In [1]:
%run ../setup_db.ipynb
conf = SandboxConfig(EXTERNAL_HOST_NAME="127.0.0.1", HOST_PORT=8563, BUCKETFS_PORT=2580)
setup_schema(conf)

%run 00_setup.ipynb
setup_cloud_storage_extension(conf)

Schema created in 70.73ms
Jar for version 2.7.6 already exists in exasol-cloud-storage-extension-2.7.6.jar, skip downloading
Jar file is already present in the bucketfs
